In [2]:
import tensorflow as tf
from sklearn.datasets import load_digits
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [6]:
digits = load_digits()
X = digits.data.astype(np.float32).reshape(-1,8,8,1)
y = digits.target.astype(np.float32).reshape(-1,1)
y = OneHotEncoder().fit_transform(y).todense()
print(X.shape)
print(y.shape)

(1797, 8, 8, 1)
(1797, 10)


In [59]:
def batchGenerator(X,y,batch_size=64):
    i = 0
    while True:
        yield X[i*batch_size:(i+1)*batch_size],y[i*batch_size:(i+1)*batch_size]
        i += 1
        if i>= X.shape[0]//batch_size:
            i=0

In [60]:
batch_generator = batchGenerator(X,y)

In [32]:
tf.reset_default_graph()
# Input laye
x_ = tf.placeholder(tf.float32,[None,8,8,1])
y_ = tf.placeholder(tf.float32,[None,10])

In [33]:
filter_number = 10
filter_size = [3,3,1]
conv_filter_w1 = tf.Variable(tf.random_normal([filter_size[0], filter_size[1], filter_size[2], filter_number]))
conv_filter_b1 = tf.Variable(tf.random_normal([filter_number]))
conv_act1 = tf.nn.relu(tf.nn.conv2d(x_, conv_filter_w1,strides=[1, 1, 1, 1], padding='SAME') + conv_filter_b1)

In [34]:
max_pool1 = tf.nn.max_pool(conv_act1,ksize=[1,2,2,1],strides=[1,1,1,1],padding="SAME")

In [35]:
filter_number = 5
filter_size = [3,3,10]
conv_filter_w2 = tf.Variable(tf.random_normal([filter_size[0], filter_size[1], filter_size[2], filter_number]))
conv_filter_b2 = tf.Variable(tf.random_normal([filter_number]))
conv_act2 = tf.nn.relu(tf.nn.conv2d(max_pool1, conv_filter_w2,strides=[1, 2, 2, 1], padding='SAME') + conv_filter_b2)

In [36]:
batch_mean, batch_var = tf.nn.moments(conv_act2, [0, 1, 2], keep_dims=True)
BN_out = tf.nn.batch_normalization(conv_act2,
                                   batch_mean,
                                   batch_var,
                                   tf.Variable(tf.zeros([5])),
                                   tf.Variable(tf.ones([5])),1e-3)

In [37]:
conv_act2 = tf.nn.relu(BN_out)

In [38]:
max_pool2 = tf.nn.max_pool(conv_act2,ksize=[1,2,2,1],strides=[1,1,1,1],padding="SAME")

In [40]:
flat = tf.reshape(max_pool2, [-1, int(max_pool2.shape[1]*max_pool2.shape[2]*max_pool2.shape[3])])

In [41]:
flat.shape

TensorShape([Dimension(None), Dimension(80)])

In [42]:
num_node = 512
w1 = tf.Variable(tf.random_normal([int(flat.shape[1]),num_node]))
b1 = tf.Variable(tf.random_normal([num_node]))
h1 = tf.nn.relu(tf.matmul(flat, w1) + b1)

In [45]:
num_node = 128
w2 = tf.Variable(tf.random_normal([int(h1.shape[1]),num_node]))
b2 = tf.Variable(tf.random_normal([num_node]))
h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)

In [50]:
out_w1 = tf.Variable(tf.random_normal([num_node,y.shape[1]]))
out_b1 = tf.Variable(tf.random_normal([y.shape[1]]))
pred = tf.nn.softmax(tf.matmul(h2,out_w1)+out_b1)

In [67]:
loss = tf.contrib.keras.losses.categorical_crossentropy(y_,pred)

In [62]:
train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

In [54]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_,1),tf.argmax(pred,1)),tf.float32))

In [68]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [69]:
epoch = 1000
for i in range(epoch): # 迭代1000个周期
    batch_x,batch_y = batch_generator.__next__()
    sess.run(train_step,feed_dict={x_:batch_x,y_:batch_y})
    if(i%100==0):
        res = sess.run(accuracy,feed_dict={x_:batch_x,y_:batch_y})
        print (i,res)


0 0.140625
100 0.078125
200 0.109375
300 0.078125
400 0.140625
500 0.125
600 0.140625
700 0.140625
800 0.078125
900 0.109375


In [73]:
sess.run(pred,feed_dict={x_:X[0:15]})

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [74]:
y

matrix([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  1.,  0.]])